In [1]:
import numpy as np
from trust_constr import minimize, NonlinearConstraint, LinearConstraint, Bounds, check_grad

## Example 1: Nonlinear Inequality Constraint
Example 15.1 from [1]

Solve:
$$\min_{x,y} f(x,y)=\frac{1}{2}(x-2)^2+\frac{1}{2}(y-\frac{1}{2})^2$$
Subject to:
$$(x+1)^{-1}-y-\frac{1}{4}\ge0$$
$$x\ge0$$
$$x\ge0$$

Solution: $$(x,y) = (1.953, 0.089)$$


First solve without defining gradient (finite difference gradient will be used):

In [2]:
def objective(x):
    return 0.5*(x[0]-2)**2+0.5*(x[1]-0.5)**2

def ineq_constraint(x):
    return 1/(x[0]+1)-x[1]-0.25

constraints = NonlinearConstraint(ineq_constraint, 0, np.inf)
bounds = Bounds([0,0], [np.inf, np.inf])

x0 = np.array([5.0, 1.0])

res = minimize(objective, x0, bounds=bounds, constraints=constraints)

print("Solution =", res.x)
print(f"Obtained using {res.nfev} objctive function evaluations.")

Solution = [1.95282327 0.08865882]
Obtained using 42 objctive function evaluations.


Now define the gradient for objective and constraint and check gradients:

In [6]:
def objective_gradient(x):
    return np.array([(x[0]-2), (x[1]-0.5)])

def ineq_gradient(x):
    return np.array([-1/((x[0]+1)**2), -1])

# check gradients against finite difference gradient
# use 5 random points
for x in np.random.uniform(low=[0,0], high=[10,10], size=(5,2)):
    print("objective difference: ", check_grad(objective, objective_gradient, x))
    print("constraint difference:", check_grad(ineq_constraint, ineq_gradient, x))



objective difference:  4.584831317952022e-07
constraint difference: 3.308232434523495e-09
objective difference:  2.254821732678175e-08
constraint difference: 2.6925162655011547e-08
objective difference:  2.456840921345611e-08
constraint difference: 5.281618465496907e-09
objective difference:  8.68988845289713e-08
constraint difference: 3.0079458535059445e-08
objective difference:  1.0467888640396644e-07
constraint difference: 1.0502792330280286e-08


Finally, minimize using the gradient functions that were just test:

In [4]:
constraints = NonlinearConstraint(ineq_constraint, 0, np.inf, jac=ineq_gradient)

res = minimize(objective, x0, jac=objective_gradient, bounds=bounds, constraints=constraints)

print("Solution =", res.x)
print(f"Obtained using {res.nfev} objctive function evaluations.")

Solution = [1.95282328 0.08865881]
Obtained using 14 objctive function evaluations.


## Example 2: Nonlinear Equality Constraint


## References
[1] Nocedal, Jorge, and Stephen J. Wright. *Numerical Optimization*. 2nd ed. Springer Series in Operations Research. New York: Springer, 2006.
